<a href="https://colab.research.google.com/github/AhmedFarrukh/DeepLearning-EdgeComputing/blob/main/notebooks/CPU_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we test the inference times of quantized models, and their original versions, on a CPU.

## **Prepare the Resource**
This notebook will try to reserve the compute_cascadelake_r device available on CHI@UC.

### **Check Availability**
Before you begin, you should check the host calendar at https://chi.uc.chameleoncloud.org/project/leases/calendar/host/ to see what node types are available.

### **Chameleon Configuration**
You can change your Chameleon project name (if not using the one that is automatically configured in the JupyterHub environment) and the site on which to reserve resources (depending on availability) in the following cell.

If you need to change the details of the Chameleon server, e.g. use a different edge device (NODE_TYPE), or a different node type depending on availability, you can also do that in the following cell.

In [ ]:
import chi, os, time
from chi import lease
from chi import server

PROJECT_NAME = os.getenv('OS_PROJECT_NAME') # change this if you need to
chi.use_site("CHI@UC")
chi.set("project_name", PROJECT_NAME)
username = os.getenv('USER') # all exp resources will have this prefix

In [ ]:
chi.set("image", "CC-Ubuntu20.04")
NODE_TYPE = "compute_cascadelake_r"
expname = "cpu-inference"

In [ ]:
res = []
lease.add_node_reservation(res, node_type=NODE_TYPE, count=1)
lease.add_fip_reservation(res, count=1)
start_date, end_date = lease.lease_duration(days=0, hours=8)

l = lease.create_lease(f"{username}-{NODE_TYPE}", res, start_date=start_date, end_date=end_date)
l = lease.wait_for_active(l["id"])  #Comment this line if the lease starts in the future

In [ ]:
# continue here, whether using a lease created just now or one created earlier
l = lease.get_lease(f"{username}-{NODE_TYPE}")
l['id']

### **Provisioning Resources**
This cell provisions resources. It will take approximately 15 minutes. You can check on its status in the Chameleon web-based UI: https://chi.uc.chameleoncloud.org/project/instances/, then come back here when it is in the READY state.

In [ ]:
reservation_id = lease.get_node_reservation(l["id"])
server.create_server(
    f"{username}-{NODE_TYPE}",
    reservation_id=reservation_id,
    image_name=chi.get("image")
)
server_id = server.get_server_id(f"{username}-{NODE_TYPE}")
server.wait_for_active(server_id)


Associate an IP address with this server:

In [ ]:
reserved_fip = lease.get_reserved_floating_ips(l["id"])[0]
server.associate_floating_ip(server_id,reserved_fip)


and wait for it to come up:

In [ ]:
server.wait_for_tcp(reserved_fip, port=22)

### **Install Basic Packages**

In [ ]:
from chi import ssh
node = ssh.Remote(reserved_fip)

In [ ]:
node.run('sudo apt update')
node.run('sudo apt -y install python3-pip python3-dev')
node.run('sudo pip3 install --upgrade pip')

#### **Install Python Packages**

In [ ]:
node.run('python3 -m pip install --user tensorflow')
node.run('python3 -m pip install --user matplotlib')
node.run('python3 -m pip install --user pathlib')
node.run('python3 -m pip install --user numpy')

### **Retrieve Materials**
Finally, get a copy of the code you will run:

In [ ]:
node.run('git clone https://github.com/AhmedFarrukh/experimental.git')

### **Run Experiment**

Verify that the code files have correctly been loaded:

In [ ]:
node.run('ls ./experimental')

Run the following cell to load CNN models and apply Dynamic Range Quantization. Both original and quantized models are saved in the ./tflite_models directory.

In [ ]:
node.run('python3 ./experimental/quantizingModels.py')

Run the next cell to load the benchmark.

In [ ]:
node.run('mkdir ./benchmark')
node.run('wget https://storage.googleapis.com/tensorflow-nightly-public/prod/tensorflow/release/lite/tools/nightly/latest/linux_x86-64_benchmark_model -P ./benchmark')
node.run('chmod +x ./benchmark/linux_x86-64_benchmark_model')

Finally, use the benchmark to measure the inference time and memory footprint of each model.

In [ ]:
node.run('python3 ./experimental/measuringInferenceTimes.py')

Check if the plots for the results have been correctly saved:

In [ ]:
node.run('ls ./results')

Copy and paste the output of the following command in the Jupyter notebook terminal to transfer the plots of results to the Jupyter environment.

In [ ]:
print(f'scp -ri ~/.ssh/id_rsa_chameleon cc@{reserved_fip}:/home/cc/results ./work')

The plots resulting from the experiment should not be in the /work/results directory of the Jupyter environment.

## **Release Resources**
If you finish with your experimentation before your lease expires, release your resources and tear down your environment by running the following (commented out to prevent accidental deletions).

This section is designed to work as a "standalone" portion - you can come back to this notebook, ignore the top part, and just run this section to delete your reasources.

In [ ]:

# setup environment - if you made any changes in the top part, make the same changes here
import chi, os
from chi import lease, server

PROJECT_NAME = os.getenv('OS_PROJECT_NAME')
chi.use_site("CHI@UC")
chi.set("project_name", PROJECT_NAME)


lease = chi.lease.get_lease(f"{username}-{NODE_TYPE}")

In [ ]:
DELETE = True
# DELETE = True

if DELETE:
    # delete server
    server_id = chi.server.get_server_id(f"{username}-{NODE_TYPE}")
    chi.server.delete_server(server_id)

    # release floating IP
    reserved_fip =  chi.lease.get_reserved_floating_ips(lease["id"])[0]
    ip_info = chi.network.get_floating_ip(reserved_fip)
    chi.neutron().delete_floatingip(ip_info["id"])

    # delete lease
    chi.lease.delete_lease(lease["id"])